In [5]:
# system imports
import os
import sys

# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

# signal processing
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz


# misc
import warnings

import glob

##
import pytz
import datetime as dt
import math
import seaborn as sns

import pickle

In [6]:

#style settings
sns.set(style='whitegrid', rc={'axes.facecolor': 'white'})

#sns.set_style({'font.family':'Arial', 'font.serif':'Times New Roman'})

In [7]:
def timedomain(rr):
    results = {}

    hr = 60000/rr
    
    results['Mean RR (ms)'] = np.mean(rr)
    results['STD RR/SDNN (ms)'] = np.std(rr)
    results['Mean HR (Kubios\' style) (beats/min)'] = 60000/np.mean(rr)
    results['Mean HR (beats/min)'] = np.mean(hr)
    results['STD HR (beats/min)'] = np.std(hr)
    results['Min HR (beats/min)'] = np.min(hr)
    results['Max HR (beats/min)'] = np.max(hr)
    results['RMSSD (ms)'] = np.sqrt(np.mean(np.square(np.diff(rr))))
    results['NNxx'] = np.sum(np.abs(np.diff(rr)) > 50)*1
    results['pNNxx (%)'] = 100 * np.sum((np.abs(np.diff(rr)) > 50)*1) / len(rr)
    return results

In [8]:
def get_non_nan_values_cols(p_df, col_name):
    
    slic_ind=p_df[col_name].isnull()
    list_ne=slic_ind.tolist()
    false_indices = [i for i in range(len(list_ne)) if not list_ne[i]]
    
    list_col_interest=p_df[col_name].to_list()
    
    non_values_list = []
    
    for false_ind in false_indices:
        non_val = list_col_interest[false_ind]
        non_values_list.append(non_val)
    
    return non_values_list


def get_file_names_from_ind_list(files_list, inds_list):
    
    selected_files_list=[files_list[sel_index] for sel_index in inds_list]
    
    return selected_files_list

In [9]:
def select_pickled_condition_files(path_pickled_files_list, cond_file_names_list):
    selected_file_cond_list = []
    
    for selected_file_path in path_pickled_files_list:
        dir_path, file_name=os.path.split(selected_file_path)
        sub_id = file_name[0:12]
    
        for cond_file in cond_file_names_list:
        
            if sub_id==cond_file:
            
                selected_file_cond_list.append(selected_file_path)
                #print(selected_file_path)
                
    return selected_file_cond_list

In [10]:
def min_max_norm(all_rr_one_list, all_rr_sublist):
    
    mean_norm_list =[]
    std_norm_list = []
    
    mean_norm_bpm_list = []
    std_norm_bpm_list = []
    
    all_rr_one_list = np.array(all_rr_one_list)
    max_val = np.max(all_rr_one_list)
    min_val =np.min(all_rr_one_list)
    
    range_list = max_val-min_val
    
    ####-------beats per minutes--------
    bpm_one_list = 60000/all_rr_one_list
    
    max_bpm = np.max(bpm_one_list)
    
    min_bpm = np.min(bpm_one_list)
    
    range_bpm =  max_bpm - min_bpm
    
    
    for sel_list in all_rr_sublist:
        
        sel_list_np = np.array(sel_list)
        
        
        
        sel_lis_np_sub = sel_list_np - min_val
        
        sel_lis_norm= sel_lis_np_sub/range_list
        
        mean_norm = np.mean(sel_lis_norm)
        mean_norm_list.append(mean_norm)
        
        std_norm  = np.std(sel_lis_norm)   
        std_norm_list.append(std_norm)
        
        ##------------for normalized BPM----------
        
        sel_list_bpm = 60000/sel_list_np
        
        
        
        sel_bpm_min_sub = sel_list_bpm - min_bpm
        
        sel_bpm_list_norm =  sel_bpm_min_sub/ range_bpm
        
        mean_norm_bpm = np.mean(sel_bpm_list_norm)
        
        std_norm_bpm = np.std(sel_bpm_list_norm)
        
        mean_norm_bpm_list.append(mean_norm_bpm)
        
        std_norm_bpm_list.append(std_norm_bpm)
        
        
        
        
        #print(mean_norm)
        #print(std_norm)
                
        
    return mean_norm_list, std_norm_list, mean_norm_bpm_list, std_norm_bpm_list

In [11]:
def split_bpm(mean_bpm_list):
    
    base_line = mean_bpm_list[0]
    tsst  = np.mean(mean_bpm_list[1:4])
    relax = np.mean(mean_bpm_list[4:])
    
    #tsst  = max(mean_bpm_list[1:4])
    #relax = min(mean_bpm_list[4:])

    new_array = [base_line,tsst, relax]
    
    return new_array

In [12]:
def get_cumsum_cond(input_array):
    
    mean_diff = np.diff(input_array)
    
    mean_diff = np.insert(mean_diff, [0], 0)
    
    mean_diff_cumsum = np.cumsum(mean_diff, dtype=float)
    
    return (mean_diff_cumsum)

In [19]:

def calc_rate_change_percent(cond_phy_array):
    
    prev_val = cond_phy_array[1]
    curr_val = cond_phy_array[2]
    change_val=((curr_val-prev_val)/abs(prev_val))
    change_percent = change_val*100
    #print(change_percent)
    
    return change_percent

In [26]:
def from_rr_to_time_anal(selected_path):
                 
    with open(selected_path, 'rb') as f:
        ibi_data = pickle.load(f)
                    
                    
    subject_id = ibi_data["subject_id"]
    event_interst = ibi_data["event_interst"]
    all_rr_one_list = ibi_data["all_rr_one_list"]
    all_rr_sublists = ibi_data["all_rr_sublists"]
            
    max_rr = np.max(all_rr_one_list)
    #print(max_rr)
    min_rr = np.min(all_rr_one_list)
    #print(min_rr)
    range_rr = max_rr - min_rr    
            
    mean_rr_cumm_list = []
    std_rr_cumm_list  = []
    mean_hr_list = []
    std_hr_list = []
            
    for index, selected_event in enumerate(event_interst):
        sel_rr = all_rr_sublists[index]
                
        #### heart rate from RR ineterval
        hr_from_rr = 60000/sel_rr
        mean_hr = np.mean(hr_from_rr)
        std_hr = np.std(hr_from_rr)
        mean_hr_list.append(mean_hr)
        std_hr_list.append(std_hr)
                
        ##### mean and std rr
        mean_rr = np.mean(sel_rr)     
        std_rr = np.std(sel_rr)
        mean_rr_cumm_list.append(mean_rr)
        std_rr_cumm_list.append(std_rr)
                
                
    mean_norm_hrv_list, std_norm_hrv_list,  mean_norm_bpm_list, std_norm_bpm_list = min_max_norm(all_rr_one_list, all_rr_sublists)
            
    export_dict= {"subject_id":subject_id , 'event_interst': event_interst, "mean_bpm": mean_hr_list, "std_bpm":std_hr_list , 
                              "mean_ibi":mean_rr_cumm_list , "std_ibi": std_rr_cumm_list , "norm_mean_ibi":mean_norm_hrv_list,  "norm_std_ibi": std_norm_hrv_list,
                             "norm_mean_bpm":mean_norm_bpm_list, "norm_std_bpm":  std_norm_bpm_list, "rmssd": ibi_data["rmssd"]}
            
            
    return export_dict

In [27]:
hrv_data_dir = "/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_final_report"


In [38]:
dest_path = "/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_final_report/from_pkl_to_csv"
#designated_folder = "hrv_analysis"
#comp_dest_path = os.path.join(dest_path, designated_folder)

isexist = os.path.exists(dest_path)

#if not isexist:
#    os.makedirs(comp_dest_path)
#    print("The new directory is created")

In [28]:
hrv_files_list = glob.glob(hrv_data_dir+ "/*.pkl")

In [39]:
for sel_file in hrv_files_list:
    fol_path, file_name_ii=os.path.split(sel_file)
    file_name_ii = file_name_ii[:-4]
    hr_data_dict = from_rr_to_time_anal(sel_file)
    
    csv_time_domain_feat_filename = file_name_ii + ".csv"
    dest_path_csv = os.path.join(dest_path,csv_time_domain_feat_filename)
    print(dest_path_csv)
    df_export = pd.DataFrame.from_dict(hr_data_dict)
    df_export.to_csv(dest_path_csv)
    
    
    
    

/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_final_report/from_pkl_to_csv/VP015_161123_DigiRelax_Experiment_2023-11-16_09h47.53.124.csv
/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_final_report/from_pkl_to_csv/VP065_050124_DigiRelax_Experiment_2024-01-05_14h41.28.548.csv
/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_final_report/from_pkl_to_csv/VP004_081123_DigiRelax_Experiment_2023-11-08_10h28.36.317.csv
/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_final_report/from_pkl_to_csv/VP26_271123_DigiRelax_Experiment_2023-11-27_13h16.52.357.csv
/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_final_report/from_pkl_to_csv/VP030_291123_DigiRelax_Experiment_2023-11-29_09h54.53.886.csv
/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_final_report/from_pkl_to_csv/VP075_180124_DigiRelax_Experiment_2024-01-18_09h54.07.734.csv
/home

In [40]:
pd.read_csv("/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_final_report/from_pkl_to_csv/Vp046_081223_DigiRelax_Experiment_2023-12-08_14h49.59.665.csv")

,Unnamed: 0,subject_id,event_interst,mean_bpm,std_bpm,mean_ibi,std_ibi,norm_mean_ibi,norm_std_ibi,norm_mean_bpm,norm_std_bpm,rmssd
0,0,Vp046_081223_DigiRelax_Experiment_2023-12-08_1...,baseline_instruction_ts 3 min,70.706429,4.879362,852.539062,57.618133,0.778716,0.099664,0.108167,0.055491,41.909685
1,1,Vp046_081223_DigiRelax_Experiment_2023-12-08_1...,tsst_prep_ts 5 min,96.001217,15.688834,641.896215,104.298593,0.414361,0.180408,0.395833,0.178422,47.378853
2,2,Vp046_081223_DigiRelax_Experiment_2023-12-08_1...,tsst_pres_ts 5 min,134.948407,6.246970,445.607980,21.510666,0.074835,0.037208,0.838762,0.071044,9.924140
3,3,Vp046_081223_DigiRelax_Experiment_2023-12-08_1...,tsst_pres_ts 10 min,142.344215,3.761552,421.812180,11.312738,0.033675,0.019568,0.922871,0.042778,5.928080
4,4,Vp046_081223_DigiRelax_Experiment_2023-12-08_1...,relaxation_prep_ts 5 min,75.802222,6.792926,797.750000,69.727072,0.683946,0.120609,0.166119,0.077253,50.880376
5,5,Vp046_081223_DigiRelax_Experiment_2023-12-08_1...,relaxation_prep_ts 10 min,75.447043,4.744253,798.334649,49.062043,0.684957,0.084864,0.162080,0.053954,36.211450
6,6,Vp046_081223_DigiRelax_Experiment_2023-12-08_1...,relaxation_prep_ts 15 min,76.041837,6.237600,794.177083,62.937310,0.677766,0.108865,0.168844,0.070937,46.055337
